<a href="https://colab.research.google.com/github/dayoungcho/dacon/blob/main/%EA%B9%8C%EC%8A%A4/%EC%A0%84%EB%A0%A5%EA%B1%B0%EB%9E%98%EC%86%8C_TFT%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [3]:
pip install pytimekr

In [4]:
pip install pytorch_forecasting

In [5]:
import pandas as pd
import datetime as dt
import itertools
from pytimekr import pytimekr
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor # Changed for consistency
from pytorch_forecasting import TemporalFusionTransformer, QuantileLoss
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
electrade_2013 = pd.read_csv('2013_2016 일별 시간대별 연료원별 전력거래량.csv', encoding='cp949')
electrade_2017 = pd.read_csv('2017~2020 일별 시간대별 연료원별 전력거래량.csv', encoding='cp949')
electrade_2021 = pd.read_csv('일별 시간대별 연료원별 거래량_20220430.csv', encoding='cp949')
electrade_2022 = pd.read_csv('한국전력거래소_연료원별 전력거래량_20231031.csv', encoding='cp949')
electrade_2023 = pd.read_csv('한국전력거래소_연료원별 전력거래량_20231231.csv', encoding='cp949')


In [7]:
# 데이터 병합

electrade_2013.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2017.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2021 = electrade_2021[['거래일', '거래시간', '연료원', '전력거래량']]
electrade_2022 = electrade_2022[['거래일', '거래시간', '연료원', '전력거래량(MWh)']]
electrade_2022.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2023 = electrade_2023[['거래일', '거래시간', '연료원', '전력거래량(MWh)']]
electrade_2023.columns = ['거래일', '거래시간', '연료원', '전력거래량']

electrade = pd.concat([electrade_2013, electrade_2017, electrade_2021, electrade_2022, electrade_2023]).reset_index(drop=True)

In [14]:
electrade

,거래일,거래시간,연료원,전력거래량
0,2013-01-01,0,원자력,16232.160050
1,2013-01-01,0,석탄,23638.589390
2,2013-01-01,0,LNG,13380.994560
3,2013-01-01,0,유류,697.044902
4,2013-01-01,0,양수,855.181596
...,...,...,...,...
1658011,2023-12-31,20,기타,42.000000
1658012,2023-12-31,21,기타,42.000000
1658013,2023-12-31,22,기타,41.000000
1658014,2023-12-31,23,기타,39.000000


In [8]:
# 공휴일 리스트

holidays = []
for i in range(2013,2024):
  holidays.append(pytimekr.holidays(i))
holidays = list(itertools.chain(*holidays))

In [9]:
electrade.loc[electrade['거래시간'] == 24, '거래시간'] = 0   # 자정 거래시간이 0이랑 24로 중복 표기되어있음 -> 0으로 통일
electrade.거래일 = pd.to_datetime(electrade.거래일)

# Combine date and hour to create a full datetime timestamp for accurate time indexing
electrade['full_timestamp'] = electrade['거래일'] + pd.to_timedelta(electrade['거래시간'], unit='h')

# Sort data to ensure correct chronological order within each group (연료원)
electrade = electrade.sort_values(by=['연료원', 'full_timestamp']).reset_index(drop=True)

# Create a numerical time index (integer) as required by TimeSeriesDataSet
# This calculates the number of hours from the earliest timestamp within each fuel_type group.
# We use dt.total_seconds() / 3600 to get hours and then convert to int.
electrade['time_idx_numeric'] = electrade.groupby('연료원')['full_timestamp'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600).astype(int)

electrade['월'] = electrade.거래일.dt.month.astype(str)
electrade['요일'] = electrade.거래일.dt.weekday.astype(str)
electrade['공휴일여부'] = electrade.거래일.isin(holidays)*1
electrade['공휴일여부'] = electrade['공휴일여부'].astype(str)
electrade['거래시간'] = electrade['거래시간'].astype(str)
# Rename columns to match the desired input for TimeSeriesDataSet and general clarity
electrade.rename(columns={
    '거래일': 'date_original', # Keeping original date for context, if needed
    '거래시간': 'hour',
    '연료원': 'fuel_type',
    '전력거래량': 'volume',
    'time_idx_numeric': 'time_idx' # This is the integer time index for TimeSeriesDataSet
}, inplace=True)

# Drop the 'full_timestamp' column as it's no longer needed after creating 'time_idx'
# And reorder columns for consistency
electrade.columns = ['date_original', 'hour', 'fuel_type', 'volume', 'full_timestamp', 'time_idx', 'month', 'day_of_week', 'is_holiday']

electrade

/tmp/ipython-input-2936656803.py:17: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  electrade['공휴일여부'] = electrade.거래일.isin(holidays)*1


,date_original,hour,fuel_type,volume,full_timestamp,time_idx,month,day_of_week,is_holiday
0,2017-01-01,0,IGCC,0.0,2017-01-01 00:00:00,0,1,6,1
1,2017-01-01,1,IGCC,0.0,2017-01-01 01:00:00,1,1,6,1
2,2017-01-01,2,IGCC,0.0,2017-01-01 02:00:00,2,1,6,1
3,2017-01-01,3,IGCC,0.0,2017-01-01 03:00:00,3,1,6,1
4,2017-01-01,4,IGCC,0.0,2017-01-01 04:00:00,4,1,6,1
...,...,...,...,...,...,...,...,...,...
1658011,2023-12-31,19,해양에너지,188.0,2023-12-31 19:00:00,61339,12,6,0
1658012,2023-12-31,20,해양에너지,186.0,2023-12-31 20:00:00,61340,12,6,0
1658013,2023-12-31,21,해양에너지,146.0,2023-12-31 21:00:00,61341,12,6,0
1658014,2023-12-31,22,해양에너지,83.0,2023-12-31 22:00:00,61342,12,6,0


In [10]:
# 모델 설정
max_prediction_length = 168 # 향후 7일(168시간)을 예측 (decoder 길이)
max_encoder_length = 336   # 과거 14일을 참조 (encoder 길이)
training_cutoff = electrade.time_idx.max() - max_prediction_length # 학습 데이터 자르는 시점

training = TimeSeriesDataSet(
    electrade[lambda x: x.time_idx <= training_cutoff],   # 학습용 데이터만 필터링

    # ---기본 설정---
    time_idx = "time_idx",
    target = "volume",
    group_ids= ["fuel_type"],

    # ---시퀀스 길이 설정---
    min_encoder_length = max_encoder_length//2,     # 최소 참조 기간(가변적일 수 있음)
    max_encoder_length = max_encoder_length,        # 최대 참조 기간
    min_prediction_length = 1,                      # 최소 예측 기간
    max_prediction_length = max_prediction_length,  # 최대 예측 기간

    # ---변수 매핑(중요)---

    # 1. 정적 변수(static)
    static_categoricals = ['fuel_type'],

    # 2. 미래를 아는 변수(known inputs)
    time_varying_known_categoricals = ['hour', 'day_of_week', 'month', 'is_holiday'],
    time_varying_known_reals = ['time_idx'],   # time_idx 여기에 포함시키기

    # 3. 과거만 아는 변수
    # time_varying_unknown_categoricals = [],
    time_varying_unknown_reals = ['volume'],  # 타겟도 포함시켜야함

    # ---전처리 옵션---
    target_normalizer = GroupNormalizer(
        groups=["fuel_type"], transformation="softplus"
    ),
    add_relative_time_idx = True,  # 시퀀스 내 상대적 시간 인덱스 자동 추가
    add_target_scales = True,      # 타겟의 스케일 정보 추가(정규화 복원 시 필요)
    add_encoder_length = True,     # 인코더 길이 정보 추가
)





In [11]:
# 실제 사용을 위한 Dataloader 변환
batch_size = 512  # GPU 메모리에 맞춰 조절
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)

In [12]:
# -----------------------------
# 1단계: DataLoader 만들기
# -----------------------------

# 검증 데이터셋(validation set) 분리: 마지막 시간을 기준으로 나눔
validation = TimeSeriesDataSet.from_dataset(
    training, electrade, predict=True,
    stop_randomization=True
)

# 학습용 로더
train_dataloader = training.to_dataloader(
    train = True,
    batch_size = batch_size,
    num_workers = 4
)

# 검증용 로더
val_dataloader = validation.to_dataloader(
    train = False,
    batch_size = batch_size * 10,
    num_workers = 0
)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [18]:
# -----------------------------
# 2단계: TFT 모델 생성
# -----------------------------

tft = TemporalFusionTransformer.from_dataset(
    training,
    # [학습 파라미터]
    learning_rate = 0.03, # 0.01~0.05 사이로 설정
    hidden_size = 64, # 모델의 크기(데이터가 적으면 16, 많으면 64-128)
    attention_head_size = 1, # 어텐션 헤드 수(보통 1~4)
    dropout = 0.0, # 과적합 방지(0.1~0.3)
    hidden_continuous_size = 8, # 연속형 변수 처리 크기

    # [손실 함수] -> tft는 확률적 예측을 하므로 QuantileLoss 사용
    loss = QuantileLoss(),

    # [로그 및 최적화 설정]
    log_interval = 10,
    optimizer = "Adam",  # # tft 저자가 추천함..아니면 torch.optim.Adam
    reduce_on_plateau_patience = 4, # 성능이 안 오르면 학습률을 줄임
)


print(f"모델 파라미터 개수: {tft.size()/1e3:.1f}k")

모델 파라미터 개수: 231.0k


/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [21]:
# -----------------------------
# 3단계: 트레이너(Trainer) 설정 및 학습 시작
# -----------------------------

# 과적합 방지: 검증 성능이 10번 연속 좋아지지 않으면 조기 종료
early_stop_callback = EarlyStopping(
    monitor = "val_loss",
    min_delta = 1e-4,
    patience = 10,
    verbose = False,
    mode = "min"
)

lr_logger = pl.callbacks.LearningRateMonitor()  # 학습률 변화 기록

trainer = pl.Trainer(
    max_epochs = 15,   # 최대 30번 학습(얼리스타핑으로 일찍 끝날 수 있음)
    accelerator = "gpu",  # gpu가 있으면 자동 사용, 없으면 cpu 사용
    devices = 1,   # 사용할 장치 개수
    enable_model_summary = True,
    gradient_clip_val = 0.1, # 학습 안정화를 위해 기울기 자르기
    callbacks = [lr_logger, early_stop_callback]
)

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores


In [ ]:
# 학습 시작

print("학습을 시작합니다...")
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


학습을 시작합니다...


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃    ┃ Name                               ┃ Type                            ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0  │ loss                               │ QuantileLoss                    │      0 │ train │     0 │
│ 1  │ logging_metrics                    │ ModuleList                      │      0 │ train │     0 │
│ 2  │ input_embeddings                   │ MultiEmbedding                  │    655 │ train │     0 │
│ 3  │ prescalers                         │ ModuleDict                      │     96 │ train │     0 │
│ 4  │ static_variable_selection          │ VariableSelectionNetwork        │  5.3 K │ train │     0 │
│ 5  │ encoder_variable_selection         │ VariableSelectionNetwork        │  6.6 K │ train │     0 │
│ 6  │ decoder_variable_selection         │ VariableSelectionNetwork        │  4.8 K │ train │     0 │
│ 7  │ static_context_variable_selection  │ GatedResidualNetwork            │ 16.8 K │ train │     0 │
│ 8  │ static_context_initial_hidden_lstm │ GatedResidualNetwork            │ 16.8 K │ train │     0 │
│ 9  │ static_context_initial_cell_lstm   │ GatedResidualNetwork            │ 16.8 K │ train │     0 │
│ 10 │ static_context_enrichment          │ GatedResidualNetwork            │ 16.8 K │ train │     0 │
│ 11 │ lstm_encoder                       │ LSTM                            │ 33.3 K │ train │     0 │
│ 12 │ lstm_decoder                       │ LSTM                            │ 33.3 K │ train │     0 │
│ 13 │ post_lstm_gate_encoder             │ GatedLinearUnit                 │  8.3 K │ train │     0 │
│ 14 │ post_lstm_add_norm_encoder         │ AddNorm                         │    128 │ train │     0 │
│ 15 │ static_enrichment                  │ GatedResidualNetwork            │ 20.9 K │ train │     0 │
│ 16 │ multihead_attn                     │ InterpretableMultiHeadAttention │ 16.6 K │ train │     0 │
│ 17 │ post_attn_gate_norm                │ GateAddNorm                     │  8.4 K │ train │     0 │
│ 18 │ pos_wise_ff                        │ GatedResidualNetwork            │ 16.8 K │ train │     0 │
│ 19 │ pre_output_gate_norm               │ GateAddNorm                     │  8.4 K │ train │     0 │
│ 20 │ output_layer                       │ Linear                          │    455 │ train │     0 │
└────┴────────────────────────────────────┴─────────────────────────────────┴────────┴───────┴───────┘

Trainable params: 231 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 231 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 314                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will 
create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller 
than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader 
running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will 
create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller 
than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader 
running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will 
create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller 
than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader 
running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(

In [ ]:
#  Best Model 로드
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# 검증 데이터에 대해 예측 수행(mode="prediction"이면 점추정(quantile 중 0.5, 즉 중앙값 반환). mode="quantiles"면 구간추정(모든 quantile 반환))
raw_predictions, x = best_tft.predict(
    val_dataloader,
    mode = "raw",
    return_x = True
)

# 결과 시각화
best_tft.plot_prediction(x, raw_predictions, idx=0, add_loss_to_title=True)

In [ ]:
predictions, index = best_tft.predic(val_dataloader, mode='prediction', return_index = True)
results = index.copy()

results['predicted_volume'] = predictions.tolist()
print(results.head())

In [ ]:
lng_preds = results[results['fuel_type'] == 'LNG']
print('LNG 향후 24시간 거래 예측값:', lng_preds.iloc[0]['predicted_volume'])


In [ ]:
import matplotlib.pyplot as plt

target_fuel = "LNG"

best_tft.plot_prediction(
    val_dataloader,
    predictions,
    index = index[index['fuel_type']==target_fuel].index[0],
    add_loss_to_title = True
)
plt.title(f"{target_fuel} 24시간 거래 예측값")
plt.show()

In [ ]:
# 일별, 시간별 LNG를 연료로 한 전력거래량
electrade_lng = electrade[electrade['연료원'] == 'LNG']
electrade_lng.reset_index(inplace=True, drop=True)
electrade_lng.drop('연료원', inplace=True,axis=1)
electrade_lng

/tmp/ipython-input-2914467080.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  electrade_lng.drop('연료원', inplace=True,axis=1)


,거래일,거래시간,전력거래량
0,2013-01-01,24,13380.994560
1,2013-01-01,1,12092.910380
2,2013-01-01,2,10736.999880
3,2013-01-01,3,9869.339046
4,2013-01-01,4,9123.019097
...,...,...,...
96403,2023-12-31,20,18996.000000
96404,2023-12-31,21,18404.000000
96405,2023-12-31,22,17872.000000
96406,2023-12-31,23,18306.000000


In [ ]:
# 일별, 시간별 전력거래량
electrade_agg = electrade.groupby(['거래일','거래시간']).sum().reset_index()
electrade_agg.drop('연료원', inplace=True,axis=1)
electrade_agg

,거래일,거래시간,전력거래량
0,2013-01-01,1,53918.166773
1,2013-01-01,2,52207.046421
2,2013-01-01,3,51198.759433
3,2013-01-01,4,50523.000055
4,2013-01-01,5,50594.166019
...,...,...,...
96403,2023-12-31,20,60224.000000
96404,2023-12-31,21,59405.000000
96405,2023-12-31,22,58758.000000
96406,2023-12-31,23,59229.000000


In [ ]:
# 일별 LNG를 연료로 한 전력거래량

electrade_dailylng = electrade_lng.groupby(['거래일']).sum().reset_index()
electrade_dailylng.drop('거래시간', inplace=True,axis=1)
electrade_dailylng

,거래일,전력거래량
0,2013-01-01,196479.142825
1,2013-01-02,376521.963524
2,2013-01-03,440858.162990
3,2013-01-04,450037.228490
4,2013-01-05,394665.561910
...,...,...
4012,2023-12-27,449425.000000
4013,2023-12-28,468120.000000
4014,2023-12-29,455049.000000
4015,2023-12-30,370363.000000


In [ ]:
# 일별 총 전력거래량

electrade_agg_daily = electrade_agg.groupby('거래일').sum().reset_index()
electrade_agg_daily.drop('거래시간', inplace=True,axis=1)
electrade_agg_daily

,거래일,전력거래량
0,2013-01-01,1.208822e+06
1,2013-01-02,1.470775e+06
2,2013-01-03,1.569730e+06
3,2013-01-04,1.579830e+06
4,2013-01-05,1.455056e+06
...,...,...
4012,2023-12-27,1.685486e+06
4013,2023-12-28,1.674182e+06
4014,2023-12-29,1.614603e+06
4015,2023-12-30,1.457736e+06
